In [1]:
import json
import ast
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
result_df = pd.read_csv('final_data.csv')

In [5]:
result_df

,user_id,name,course_order,seq,total_videos_watched,total_watch_time,total_videos_required
0,U_112,姚保峰,"[696994, 735338, 696956, 677020, 697027, 78857...","[{'video_id': 'V_1395633', 'segment': [{'start...",6.0,584.200,3505
1,U_150,韩明瑞,"[682155, 697374]","[{'video_id': 'V_6224262', 'segment': [{'start...",1.0,220.000,178
2,U_172,刘文博,"[948410, 936990, 1448497, 1471241, 697791, 676...","[{'video_id': 'V_6432959', 'segment': [{'start...",1.0,4.700,1177
3,U_189,张强,"[927961, 697334, 817148, 682155, 746971, 94325...","[{'video_id': 'V_6334508', 'segment': [{'start...",20.0,6058.000,1546
4,U_197,李超,"[697791, 676937, 1794464, 808526, 948415, 6941...","[{'video_id': 'V_1358540', 'segment': [{'start...",55.0,11304.248,394
...,...,...,...,...,...,...,...
112374,U_26862793,NaN,"[697791, 676932]","[{'video_id': 'V_1358540', 'segment': [{'start...",1.0,60.100,171
112375,U_26863000,发发,"[676932, 697068, 696767, 780629, 697791, 69699...","[{'video_id': 'V_1395633', 'segment': [{'start...",3.0,289.000,2250
112376,U_26863054,杨智婷,"[735031, 845950, 770784, 735149, 676662]","[{'video_id': 'V_1410123', 'segment': [{'start...",3.0,665.960,675
112377,U_26863059,ZhangQJ,"[696892, 682350, 697270, 697423, 697155, 69703...","[{'video_id': 'V_6185387', 'segment': [{'start...",11.0,3915.000,796


In [8]:
selected_df = result_df.drop(columns=['name', 'course_order', 'seq'])

In [9]:
selected_df

,user_id,total_videos_watched,total_watch_time,total_videos_required
0,U_112,6.0,584.200,3505
1,U_150,1.0,220.000,178
2,U_172,1.0,4.700,1177
3,U_189,20.0,6058.000,1546
4,U_197,55.0,11304.248,394
...,...,...,...,...
112374,U_26862793,1.0,60.100,171
112375,U_26863000,3.0,289.000,2250
112376,U_26863054,3.0,665.960,675
112377,U_26863059,11.0,3915.000,796


In [11]:
selected_df['watch_ratio'] = selected_df['total_videos_watched'] / selected_df['total_videos_required']
selected_df['completion_status'] = selected_df['watch_ratio'].apply(lambda x: 1 if x > 0.4 else 0)
selected_df

,user_id,total_videos_watched,total_watch_time,total_videos_required,watch_ratio,completion_status
0,U_112,6.0,584.200,3505,0.001712,0
1,U_150,1.0,220.000,178,0.005618,0
2,U_172,1.0,4.700,1177,0.000850,0
3,U_189,20.0,6058.000,1546,0.012937,0
4,U_197,55.0,11304.248,394,0.139594,0
...,...,...,...,...,...,...
112374,U_26862793,1.0,60.100,171,0.005848,0
112375,U_26863000,3.0,289.000,2250,0.001333,0
112376,U_26863054,3.0,665.960,675,0.004444,0
112377,U_26863059,11.0,3915.000,796,0.013819,0


In [19]:
features = ['watch_ratio']
X = selected_df[features]
y = selected_df['completion_status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Áp dụng SMOTE để cân bằng lại dữ liệu
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print(f"Original training set size: {X_train.shape}, Resampled training set size: {X_train_resampled.shape}")

Original training set size: (78665, 1), Resampled training set size: (155094, 1)


In [20]:
scaler = StandardScaler()

# Chuẩn hóa dữ liệu huấn luyện
X_train_resampled = scaler.fit_transform(X_train_resampled)

# Chuẩn hóa dữ liệu kiểm tra
X_test = scaler.transform(X_test)

In [21]:
# Xây dựng mô hình neural network
model = Sequential()

# Thêm các lớp vào mô hình
model.add(Dense(64, input_dim=X_train_resampled.shape[1], activation='relu'))  # Lớp ẩn đầu tiên
model.add(Dense(32, activation='relu'))  # Lớp ẩn thứ hai
model.add(Dense(1, activation='sigmoid'))  # Lớp đầu ra (dùng sigmoid cho nhị phân)

# Biên dịch mô hình
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(X_train_resampled, y_train_resampled, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
4847/4847 [==============================] - 7s 1ms/step - loss: 0.0234 - accuracy: 0.9956 - val_loss: 0.0025 - val_accuracy: 0.9985
Epoch 2/10
4847/4847 [==============================] - 7s 1ms/step - loss: 0.0035 - accuracy: 0.9990 - val_loss: 0.0030 - val_accuracy: 0.9985
Epoch 3/10
4847/4847 [==============================] - 7s 1ms/step - loss: 0.0027 - accuracy: 0.9993 - val_loss: 0.0035 - val_accuracy: 0.9985
Epoch 4/10
4847/4847 [==============================] - 7s 1ms/step - loss: 0.0025 - accuracy: 0.9993 - val_loss: 0.0018 - val_accuracy: 0.9990
Epoch 5/10
4847/4847 [==============================] - 7s 1ms/step - loss: 0.0022 - accuracy: 0.9993 - val_loss: 0.0014 - val_accuracy: 0.9994
Epoch 6/10
4847/4847 [==============================] - 7s 1ms/step - loss: 0.0020 - accuracy: 0.9995 - val_loss: 0.0043 - val_accuracy: 0.9986
Epoch 7/10
4847/4847 [==============================] - 8s 2ms/step - loss: 0.0020 - accuracy: 0.9994 - val_loss: 3.4023e-04 - val_accur

In [22]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

1054/1054 [==============================] - 1s 1ms/step - loss: 0.0033 - accuracy: 0.9988
Test Loss: 0.0033, Test Accuracy: 0.9988


In [23]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)  # Chuyển giá trị xác suất thành nhãn 0 hoặc 1

# In ra một số kết quả dự đoán
print(f"Predictions: {y_pred[:10]}")

1054/1054 [==============================] - 1s 792us/step
Predictions: [[0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]]


In [24]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     33225
           1       0.92      1.00      0.96       489

    accuracy                           1.00     33714
   macro avg       0.96      1.00      0.98     33714
weighted avg       1.00      1.00      1.00     33714

